# Visualization Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [3]:
import time
from contextlib import contextmanager


@contextmanager
def timer():
    """Helper function to time code within a context block."""
    start_time = time.perf_counter()
    yield
    duration = time.perf_counter() - start_time
    print(f"Ran in {duration:0.1f} secs")

In [8]:
import math
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from src.pipeline.flows.main_no_prefect import main_flow
from src.visualization.interactive_play_selector import create_interactive_play_selector

In [5]:
with timer():
    _ = main_flow(max_weeks=3, max_games=3, max_plays=3)

Ran in 12.1 secs


In [6]:
df_plays_all = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_tracking_display_all = pd.read_csv(f"{DIR}/data/outputs/tracking_display.csv")
df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

In [9]:
create_interactive_play_selector(
    df_plays_all,
    df_tracking_display_all,
    df_areas_all,
    continuous_update=True
)

interactive(children=(Dropdown(description='Week', options=(1, 2, 3), value=1), Dropdown(description='Game ID'…